In [4]:
import matplotlib.pyplot as plt
import os
import cv2 as cv
import numpy as np

Steps:
Convert left view to CIELAB -> lightness is the monochrome image
Right view is the guidance image
Original left view is the ground truth

Guidance image is downsampled to half size
https://en.wikipedia.org/wiki/Bayer_filter
Green channel is "replicated" to have 2x green a red, blue. 
(Check whether RGB to Bayer is the same thing) 
cv2.cvtColor 
https://docs.opencv.org/4.x/db/d8c/group__cudaimgproc__color.html#ga7fb153572b573ebd2d7610fcbe64166e
SceneFlow -> get to later

In [6]:
os.chdir(os.getcwd() + '\All-2views')

In [7]:
base_dir = os.getcwd()
base_dir

'C:\\Users\\jpienkny\\Documents\\EECS 556\\Report\\All-2views'

In [8]:
os.listdir()

['Art',
 'Books',
 'Computer',
 'Dolls',
 'Drumsticks',
 'Dwarves',
 'Laundry',
 'Moebius',
 'Reindeer']

Let's use "Art" as a sample. Will make this more seamless down the line.

In [9]:
views_wanted = ['\Art', '\Books', '\Computer']
image_list = [] # List Format: it is n x 2, n is the number of images, [0] is left (Ground Truth), [1] is right (Guidance), [2] is mono
disp_list = [] # Same list format, disparity matrices if we need

cur_dir = base_dir
for view in views_wanted:
    new_dir = cur_dir + view
    os.chdir(new_dir)
    img_left = cv.imread('view1.png')
    assert img_left is not None, "file could not be read, check with os.path.exists()"
    img_right = cv.imread('view5.png')
    assert img_right is not None, "file could not be read, check with os.path.exists()"
    
    mono = cv.cvtColor(img_left, cv.COLOR_BGR2LAB) # Mono image
    
    image_list.append([img_left, img_right, mono[:,:,0]]) # Only take luminance image
    disp_left = cv.imread('disp1.png')
    disp_right = cv.imread('disp5.png')
    if disp_left is not None:
        disp_left = cv.cvtColor(disp_left, cv.COLOR_BGR2GRAY)
    if disp_right is not None:
        disp_right =  cv.cvtColor(disp_right, cv.COLOR_BGR2GRAY) # If an image doesn't have a disparity img, will store as None, not fail. 
    disp_list.append([disp_left, disp_right])
    cur_dir = base_dir
    
    


In [10]:
len(image_list)

3

Guidance Image Manipulation: 
Downsample to half size (both dims), then convert to bayer format, then do demosaicing interpolation/upsampling

In [11]:

test_image = image_list[0][1] # Original Guidance Image/Right Imagcal
test_disp = disp_list[0][1]
window_name = 'image'
cv.imshow(window_name, test_image)
cv.waitKey(0)

-1

In [12]:
cv.imshow(window_name, test_disp)
cv.waitKey(0)
          

-1

In [13]:
cv.destroyAllWindows()

Max Disp -> Set D

In [ ]:
plt.hist(test_disp)
plt.show()

In [ ]:
test_half = cv.resize(test_image, (0,0), fx = 0.5, fy = 0.5)

In [ ]:
test_half.shape # BGR, need BGGR

In [ ]:
test_half_g = test_half[:,:,1]

In [ ]:
cv.imshow(window_name, test_half)
cv.waitKey(0)

In [ ]:
test_half_g.shape

In [ ]:
bggr_format.shape

In [ ]:
demosaiced = cv.cvtColor(bggr_format, cv.COLOR_BayerBGGR2BGR)

In [ ]:
B = test_half[:,:,0]
B.shape

In [ ]:
B[0::2].shape

In [ ]:
bggr_format[0::,0::] = test_half[:,:,0]

In [ ]:
bggr_format[0::,0::].shape

In [15]:
def BGR2BayerBGGR(BGR): # Take in full image # Inspired from https://github.com/redpower1998/RGB2Bayer/blob/main/RGBToBayer/RGBToBayer.cpp
    rows,cols,_ = BGR.shape # 
    
    BayerBGGR = np.zeros([rows, cols])
    
    for i in range(rows):
        for j in range(cols):
            
            pix = BGR[i,j,:] # BGR Pixels
            
            if (i % 2 == 1) and (j % 2 == 1):
                BayerBGGR[i,j] = BGR[i,j,0] # Blue
            
            elif ((i % 2 == 1) and (j % 2 == 0)) or ((i % 2 == 0) and (j % 2 == 1)):
                BayerBGGR[i,j] = BGR[i,j,1] # Green
        
            else:
                BayerBGGR[i,j] = BGR[i,j,2] # Red
                
            
            
    
    
    
    
    
    
    
   
    
    return BayerBGGR


In [16]:
bayer_out = np.uint8(BGR2BayerBGGR(test_image))
bayer_out.shape

(370, 463)

In [17]:
processed_image = cv.demosaicing(bayer_out, cv.COLOR_BayerBG2BGR) # Bilinear Interpolation

In [ ]:
cv.imshow(window_name, processed_image)
cv.waitKey(0)

In [18]:
comp_image = processed_image.astype(float)

In [19]:
denois = cv.fastNlMeansDenoisingColored(processed_image)

In [20]:

cv.imshow(window_name, denois)
cv.waitKey(0)

-1

In [22]:
import colour # Evaluation Metrics

PSNR: 
https://docs.opencv.org/4.x/d5/dc4/tutorial_video_input_psnr_ssim.html#autotoc_md217

SSIM: 
https://scikit-image.org/docs/stable/api/skimage.metrics.html

In [25]:
psnr = cv.PSNR(processed_image, test_image)
psnr


33.20616926310984

AttributeError: module 'cv2' has no attribute 'SSIM'

https://python-colormath.readthedocs.io/en/latest/delta_e.html delta E

In [ ]:
double getPSNR(const Mat& I1, const Mat& I2)
{
 Mat s1;
 absdiff(I1, I2, s1); // |I1 - I2|
 s1.convertTo(s1, CV_32F); // cannot make a square on 8 bits
 s1 = s1.mul(s1); // |I1 - I2|^2
 
 Scalar s = sum(s1); // sum elements per channel
 
 double sse = s.val[0] + s.val[1] + s.val[2]; // sum channels
 
 if( sse <= 1e-10) // for small values return zero
 return 0;
 else
 {
 double mse = sse / (double)(I1.channels() * I1.total());
 double psnr = 10.0 * log10((255 * 255) / mse);
 return psnr;
 }
}

Algorithm Skeleton

Input Parameters: NOTE: We need code to make disparity maps to set Wh same as paper, but for now we can use the examples that have pregenerated maps.

In [ ]:
S = 16 # Patch size sxs
Wh = D_Test # maximum Disparity of images
Wv = 30 # Vertical Search Range
N = 5 # Using Figure 4
T = N # Paper Results

In [ ]:
L = np.zeros(test_image.shape)
U_a = np.zeros(test_image.shape)
U_b = np.zeros(test_image.shape)

In [ ]:
BGR2BayerBGGR(test_half)

In [ ]:
help cv.demosaicing()

In [ ]:
test_half.shape